<a href="https://colab.research.google.com/github/mikejin01/POSH-Event-Data-Analyzer/blob/main/Posh_event_commission_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#Developed by Mike Jin - 2025
#This scripts help you to upload your Posh Event CSV File
#and Calculate the Commission by Codes
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Now you can browse in the left sidebar (Files tab) to copy the path
# Paste that path here, e.g.:
file_path = '/content/drive/My Drive/Events/20250823-anime-rave-posh-final.csv'

# Read CSV
df = pd.read_csv(file_path)

# strip spaces from column names just in case
df.columns = df.columns.str.strip()

# Filter rows: promo code not empty & order subtotal > 0
filtered = df[(df["Promo Code"].notna()) & (df["Promo Code"] != "") & (df["Order Subtotal"] > 0)]

# Group by Promo Code and sum Order Subtotal
promo_totals = filtered.groupby("Promo Code")["Order Subtotal"].sum().reset_index()


# Sort descending to see biggest totals first
promo_totals = promo_totals.sort_values(by="Order Subtotal", ascending=False)

promo_totals["Commission"] = promo_totals["Order Subtotal"] * 0.20
promo_totals

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Promo Code,Order Subtotal,Commission
5,sojuking,395.0,79.0
0,4amwav,175.0,35.0
3,mixee,105.0,21.0
1,anime,25.0,5.0
2,estella,25.0,5.0
4,remix,10.0,2.0
